In [1]:
import cv2
import time
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
connections=list(mp_hands.HAND_CONNECTIONS)

In [3]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame1=cap.read()
    break
cap.release()
cv2.destroyAllWindows()
frame1=cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
results = hands.process(frame1)
# mp.solutions.drawing_utils.draw_landmarks(frame, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS)
# mp.solutions.drawing_utils.draw_landmarks(frame, results.multi_hand_landmarks[1], mp_hands.HAND_CONNECTIONS)

In [200]:
x=[i.x for i in results.multi_hand_landmarks[0].landmark]
y=[i.y for i in results.multi_hand_landmarks[0].landmark]
z=[i.z for i in results.multi_hand_landmarks[0].landmark]
z=[abs(i) for i in z]
y=[abs(i) for i in y]
shape=frame1.shape
x = (np.array(x) * shape[0]).astype(int)
y = (np.array(y) * shape[1]).astype(int)
z = (np.array(z) * shape[0]).astype(int)
img=np.zeros((shape[0],shape[1],3), dtype=np.uint8)
plt.imshow(frame1)

In [4]:
def plotm(x,y):
    for i in range(len(connections)):
        plt.plot([x[connections[i][0]],x[connections[i][1]]], [y[connections[i][0]],y[connections[i][1]]], 'ro-')
    plt.show()

In [5]:
def make_connections(x, y,img,color = (0, 0, 255),thickness=2):
    for connection in connections:
        start_point = (x[connection[0]], y[connection[0]])
        end_point = (x[connection[1]], y[connection[1]])
        img = cv2.line(img, start_point, end_point, color, thickness)
    return img

In [6]:
def show_image(x,y,img):
    img=make_connections(x,y,img)
    cv2.imshow("Connections", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
def modify(x,y,z,shape=(800,600)):
    x=(np.array(x) * shape[0]).astype(int)
    y=(np.array(y) * shape[1]).astype(int)
    z=(np.array(z) * shape[0]).astype(int)
    x=[abs(i) for i in x]
    y=[abs(i) for i in y]
    z=[abs(i) for i in z]
    return x,y,z

In [8]:
connections=list(mp_hands.HAND_CONNECTIONS)
shape=(800, 600)
cap = cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    frame = cv2.resize(frame, shape)
    frame = cv2.flip(frame, 1)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame)
    img1= np.zeros((shape[0],shape[1],3), dtype=np.uint8)
    img2= np.zeros((shape[0],shape[1],3), dtype=np.uint8)
    img3= np.zeros((shape[0],shape[1],3), dtype=np.uint8)
    if results.multi_hand_landmarks:
        for hand_landmark in results.multi_hand_landmarks:
            x=[]
            y=[]
            z=[]
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmark, connections)
            for landmark in hand_landmark.landmark:
                x.append(landmark.x)
                y.append(landmark.y)
                z.append(landmark.z)
            x,y,z=modify(x,y,z)
            img1=make_connections(x,y,img1)
            img2=make_connections(x,z,img2)
            img3=make_connections(z,y,img3)
    cv2.imshow('Hand Detection', frame)
    cv2.imshow('Front View', img1)
    cv2.imshow('Top View', img2)
    cv2.imshow('Side View', img3)
    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# mp.solutions.drawing_utils.draw_landmarks(frame1, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS)
plt.imshow(frame1)

In [230]:
x=[i.x for i in results.multi_hand_landmarks[0].landmark]
y=[i.y for i in results.multi_hand_landmarks[0].landmark]
z=[i.z for i in results.multi_hand_landmarks[0].landmark]

x_w=[i.x for i in results.multi_hand_world_landmarks[0].landmark]
y_w=[i.y for i in results.multi_hand_world_landmarks[0].landmark]
z_W=[i.z for i in results.multi_hand_world_landmarks[0].landmark]